# file for creating the dataframe

In [1]:
# requirements txt !

import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import dotenv
import re
import os

### get access token

In [5]:
# .env
config = dotenv.dotenv_values(".env")

PASSWORD = config["PASSWORD"]

CLIENT_SECRET = config["CLIENT_SECRET"]

token_url = 'https://accounts.kielregion.addix.io/realms/infoportal/protocol/openid-connect/token'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'grant_type': 'password',
    'username': 'justus.heilingbrunner@student.fh-kiel.de', 
    'password': PASSWORD,
    'client_id': 'quantumleap',
    'client_secret': CLIENT_SECRET
}

response = requests.post(token_url, headers=headers, data=data)

if response.status_code == 200:
    token_data = response.json()
    access_token = token_data['access_token']
    print(f"Bearer Token successful requested")
    if access_token:
        dotenv_path = '.env'
        
        dotenv.set_key(dotenv_path, 'ACCESS_TOKEN', access_token)
        print(f"Access Token successfully written to the .env file.")
else:
    print(f"Error: {response.status_code}, {response.text}")

Bearer Token successful requested
Access Token successfully written to the .env file.


### make dataframe function

In [ ]:
### functions

def fetch_station_data(station_id, from_date, to_date):
    """
    Retrieves data for a specified bike hire docking station over a given time period.

    This function connects to a specified URL to access data regarding the availability of bikes. It
    makes an HTTP GET request with authentication and specific query parameters to gather
    aggregated data across the specified dates.

    :param station_id: Unique identifier for the bike hire docking station.
    :type station_id: str

    :param from_date: The start date and time from when the data is to be fetched.
    :type from_date: datetime

    :param to_date: The end date and time until when the data is to be fetched.
    :type to_date: datetime

    :return: A JSON object containing the response data if successful, None otherwise. The JSON includes
             aggregated available bike numbers per hour. If the request fails, an error message and status
             code are printed.
    :rtype: dict or None
    
    :raises Exception: Raises a printed error with status code and text if the response is unsuccessful.
    """

    url = f"{BASE_URL}{station_id}"
    headers = {
        'NGSILD-Tenant': 'infoportal',
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    params = {
        'type': 'BikeHireDockingStation',
        'fromDate': from_date.isoformat(),
        'toDate': to_date.isoformat(),
        'attrs': 'availableBikeNumber',
        'aggrPeriod': 'hour',
        'aggrMethod': 'avg'
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print(f'Got a response for station_id: {station_id}')
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def create_dataframe_from_api_data(data):
    """
    Converts data received from an API response into a structured pandas DataFrame.

    This function parses data from a JSON-like dictionary that includes time indices,
    entity identifiers, and various attributes into a DataFrame that can be used for 
    further data analysis or visualization.

    :param data: The data received from an API request, expected to include keys like 
                 'index', 'entityId', and 'attributes' which contain measurement values.
    :type data: dict

    :return: A DataFrame with time indices, an entity id extracted from 'entityId', and columns 
             for each attribute contained within 'attributes'. This DataFrame is restructured to 
             place 'entityId' and 'time_utc' columns first.
    :rtype: pandas.DataFrame

    :raises ValueError: If essential keys such as 'index', 'entityId', or 'attributes' are missing in the data.
    """

    if not all(key in data for key in ['index', 'entityId', 'attributes']):
        raise ValueError("Data missing one of the essential keys: 'index', 'entityId', 'attributes'")
    
    # Dictionary to store attribute values
    # attribute_data = {}

    # Extract the index from the response
    time_index = pd.to_datetime(data['index'])

    # Extract entityId and entityType
    entity_id = data['entityId']

    # Extract the number after "KielRegion" from the entityId
    match = re.search(r'KielRegion:(\d+)', entity_id)
    entity_id_number = match.group(1) if match else ''  # Get the number or set to empty if not found

    # Loop through each attribute dictionary in 'attributes'
    # for attribute in data['attributes']:
    #     attr_name = attribute['attrName']
    #     attribute_data[attr_name] = attribute.get('values', [])

    # Dictionary to accumulate attribute values
    attribute_data = {attr['attrName']: attr.get('values', []) for attr in data['attributes']}
    
    # Create a pandas DataFrame from the dictionary
    df = pd.DataFrame(attribute_data)
    # Add the entityId number and index values as new columns
    df['entityId'] = entity_id_number
    df['time_utc'] = time_index

    # Reorder the columns to have 'entityId' first, then 'time', followed by the rest
    column_order = ['entityId', 'time_utc'] + [col for col in df.columns if col not in ['entityId', 'time_utc']]
    df = df[column_order]

    return df

def update_and_save_station_data(DATA_TEMP_FILENAME, STATION_IDS, START_DATE, END_DATE):
    """
    Updates and saves bike station data for a specified period.
    
    This function checks for existing data from a CSV file, updates it with new data from a specified 
    time range if data for some dates are missing, and then saves the updated data back to the CSV file.

    Parameters:
    - DATA_TEMP_FILENAME (str): The path to the data file to read from and write to.
    - STATION_IDS (list): A list of station identifiers for which data needs to be updated.
    - START_DATE (datetime): The start datetime from which to fetch new data.
    - END_DATE (datetime): The end datetime until which to fetch new data.

    The function fetches data using an API call for each station ID, checks for dates where data is missing,
    and requests data for those dates. It then combines this with existing data, sorts it, and saves
    it back to a CSV file.

    Prints messages to indicate successful data processing or if no new data were processed.

    Returns:
    None
    """

    # Prüfen, ob data_temp.csv vorhanden ist
    if os.path.exists(DATA_TEMP_FILENAME):
        # Laden des existierenden DataFrame
        old_data_temp = pd.read_csv(DATA_TEMP_FILENAME)
        # make 'time_utc' in datetime
        old_data_temp['time_utc'] = pd.to_datetime(old_data_temp['time_utc'])
        # lösche alle daten vor START_DATE
        old_data_temp = old_data_temp[old_data_temp['time_utc'] >= START_DATE]
    else:
        # Erstellen eines leeren DataFrame, wenn die Datei nicht existiert
        old_data_temp = pd.DataFrame(columns=['entityId', 'time_utc'])

    #  - timedelta(hours=1), damit der request_start_date nicht gleich END_DATE ist
    full_date_range = pd.date_range(start=START_DATE, end=END_DATE - timedelta(hours=1), freq='h') 
    # Liste von DataFrames
    dataframes = []

    for station_id in STATION_IDS:
        # überprüfe für station_id, ob der zeitraum von START_DATE bis END_DATE in old_data_temp vorhanden ist:
        # select one station
        station_data = old_data_temp[old_data_temp['entityId'] == station_id]
        # extract available dates
        available_dates = station_data['time_utc']
        # Ermitteln der fehlenden Daten
        missing_dates = full_date_range[~full_date_range.isin(available_dates)]

        # wenn ja, skip diese station_id
        # wenn nein, mache ein request_start_date

        # Daten nur für fehlende Zeiten anfordern
        if not missing_dates.empty:
            request_start_date = missing_dates[0]
            # und requeste die nicht vorhandenen stunden bis zum END_DATE
            data = fetch_station_data(station_id, request_start_date, END_DATE)
            if data:
                df = create_dataframe_from_api_data(data)
                # und appende sie an das dataframe
                dataframes.append(df)

    if dataframes:
        # Alle neuen DataFrames der Stationen zusammenführen
        new_data_temp = pd.concat(dataframes)
        # make the entitiy_id a number 
        new_data_temp['entityId'] = new_data_temp['entityId'].astype('int64')
        # Zusammenführen des alten DataFrames mit dem neuen
        combined_data_temp = pd.concat([old_data_temp, new_data_temp])
        # Sortieren, nach entitiyId und time_utc
        combined_data_temp = combined_data_temp.sort_values(by=['entityId', 'time_utc'])
        # resete index
        combined_data_temp = combined_data_temp.reset_index(drop=True)
        # DataFrame in eine CSV-Datei speichern
        combined_data_temp.to_csv(DATA_TEMP_FILENAME, index=False)

        # count new records and unique Ids 
        total_new_records = len(new_data_temp)
        unique_stations = new_data_temp['entityId'].nunique()

        print(f'{total_new_records} new records fetched for {unique_stations} stations.')
    else:
        print('No new data to process. Existing data used.')

    print('-------------')
    print(f'Data successfully loaded and saved for STATION_IDS:{STATION_IDS}')
    print(f'Time in UTC:\nStart Date: {START_DATE}\nEnd Date: {END_DATE}')

In [ ]:
### Konfigurations

config = dotenv.dotenv_values(".env")

ACCESS_TOKEN = config["ACCESS_TOKEN"]

BASE_URL = "https://apis.kielregion.addix.io/ql/v2/entities/urn:ngsi-ld:BikeHireDockingStation:KielRegion:"

STATION_IDS = [24370, 24397, 24367, 24399]  # Beispielliste von Station IDs

# API mit UTC time steps
# Calculate the end date by rounding down to the closest whole hour in UTC !,
# to make sure to get hourly averages for whole hours with API request
END_DATE = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
START_DATE = END_DATE - timedelta(days=1) # timedelta anpassen an model sliding window length (=24 hours)

DATA_TEMP_FILENAME = 'data_temp.csv'

In [10]:
### Usage

update_and_save_station_data(DATA_TEMP_FILENAME, STATION_IDS, START_DATE, END_DATE)

Got a response for station_id: 24399
24 new records fetched for 1 stations.
-------------
Data successfully loaded and saved for STATION_IDS:[24370, 24397, 24367, 24399]
Time in UTC:
Start Date: 2024-11-24 07:00:00+00:00
End Date: 2024-11-25 07:00:00+00:00


***